In [ ]:
import pandas as pd
import requests
import json

# Define the URL of your local Ollama instance
ollama_url = "http://localhost:11434/api/generate"  # Replace with your actual URL

# Path to your local CSV file
file_path = "/var/tmp/produkt_daten.csv"  # Update this path



preprompt = """Du bist ein Programm fuer Produktdaten, das Produktnamen aus bestehenden Daten zusammensetzt. 

Dazu werden dir im naechstern Schritt Daten aus einer Tabelle mit Produktdaten von Bueromoebel des Herstellers "Hammerbacher" uebergeben. 
Deine Aufgaben werden jetzt erklaert. 


Generiere einen neuen Produktnamen, der die folgenden Struktuer hat:

1. Herstellername
2. Das erste Wort von 'Verkaufsbezeichnung kurz'
2. Alle Information aus 'Verkaufsbezeichnung kurz'
3. Ersetze Abkürzungen wie z.B. "BO" mit dem Wort "Bogengriff". Die Langform befindet sich in 'Verkaufsbezeichnung lang': z.B "Bogengriff BO"
4. Der Name darf nur normalen Text enthalten, keine Abkürzungen und keine Kommas



Bestaetige, dass du das verstanden hast""" 

prompt = """Analysiere nun die folgenden Informationen und generiere einen neuen Produktnamen. 
verwende den Namen aus der Spalte 'Verkaufsbezeichnung kurz' als Start fuer die Generierung.
Als Antwort wirst nur du gefragten Text zurueckgeben ohne Erklarungen und Alternativen. 
"""





def sendRequest(prompt):
    # Create a payload for the request
    payload = {
        "model": "llama3.2:latest", 
        "prompt": prompt,
        "max_tokens": 5,  # Adjust as needed
        "temperature": 0.5,  # Adjust as needed
    }
    # Send a POST request to the Ollama instance
    response = requests.post(ollama_url, json=payload)

    # Print the raw response content
    #print("Raw response content:", response.text)
    complete_response = ""
    # Check if the request was successful
    if response.status_code == 200:
        # Iterate over each line in the response
        for line in response.iter_lines():
            if line:  # Make sure line is not empty
                try:
                    result = json.loads(line)  # Parse each JSON object
                    complete_response += result["response"]  # Concatenate responses
                except json.JSONDecodeError as e:
                    print("Error decoding JSON:", e)
        print(complete_response)
        return complete_response.strip()  # Return cleaned response

    else:
        print("Error:", response.status_code, response.text)
        return None


sendRequest(preprompt)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

for index, row in df.iterrows():
    prompt_string = []
    
    for column_name, value in row.items():
         if pd.notna(value) and value != "":
            prompt_string.append(f"{column_name}: {value}")
            if(column_name=="Verkaufsbezeichnung kurz"):
                old_name=value
             

    # Join all non-empty column values into a single string
    full_prompt = ", ".join(prompt_string)

    prompt += full_prompt
    print(prompt)
    sendRequest(prompt)

: 